<a href="https://colab.research.google.com/github/4ndrebar/LabFis1/blob/main/fit_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Istruzioni
In questo notebook troverete le funzioni necessarie per tracciare i grafici ed effettuare i fit delle acquisizioni dati del vostro esperimento. Il file è in sola lettura quindi per utilizzarlo dovrete **salvarne una copia sul vostro drive**. <br> Siete vivamente invitati a mettere le mani sul codice e personalizzarlo a vostro piacimento, ma meglio farlo in un secondo momento: **finchè siete in laboratorio approfittatene per concentrarvi sulla raccolta dati e <ins>sul corretto svolgimento dell'esperimento!</ins>**

* Importare le librerie necessarie
* Per ogni fit  da effettuare caricare il file di dati corrispondente sulla macchina virtuale di Colab tramite l'apposita cella di codice che contiene la funzione `files.upload()`
* Quando necessario effettuare più fit ricopiare la sezione di celle che importa i dati e chiama la funzione di fit (naturalmente si potrebbe fare un loop... in un secondo momento)
* Prima di caricare files con lo stesso nome eliminare quelli preesistenti per evitare conflitti.


In [ ]:
import numpy as np  
from matplotlib import pyplot as plt 
from scipy.optimize import curve_fit 
from scipy import odr
from ipywidgets import interact, interact_manual, FloatSlider, Checkbox, BoundedFloatText
from google.colab import files


## Esperimento del pendolo

### Fit sinusoidale

#### Importa i dati

In [ ]:
filename = list(files.upload().keys())[0] #chiede di caricare il file e ne estrae il nome

In [ ]:
dati = np.loadtxt(filename, dtype=np.str, delimiter="\t", skiprows=2) #legge il file formattato con tabulazioni e scarta le prime due righe di header
dati = np.char.replace(dati, ',', '.') #rimpiazza il separatore decimale
dati = dati[:,0:2]  #seleziona solo le colonne posizione e tempo
#dati[dati == ''] = 0 #rimpiazza le stringhe vuote con 0; le trovo alla fine del file quando salvo più misure nello stesso file.
dati = dati.astype(np.float)   #riconverte le entrate a float

t=dati[:,0]
y=dati[:,1]

plt.figure()    #visualizza i dati importati
_=plt.plot(t, y, marker='o')

#### Fit

In [ ]:
#Definizione della funzione di fit 
def damped_sine(t, A, omega, phi, gamma, y0): 
  return A*np.exp(-gamma*t)*np.sin(omega*t+phi)+y0

In [ ]:
# Fitting routine
# definisce la funzione che stampa il grafico e esegue il fit dei dati. È sufficiente eseguirla una volta per salvare la funzione in memoria.

 
def fitplot_sin(A, omega, y0, phi, gamma, print_output=True, plot_guess=False):
 
  guess = np.array([A,omega,phi,gamma,y0]) #array delle stime iniziali
  bounds = ([0,0,-np.pi,0,0], [10., 10, np.pi,1,2]) 
 
  global popt, pcov    #i risultati del fit sono variabili globali 
 
  plt.figure(figsize=(15, 10))
  popt, pcov = curve_fit(damped_sine, t, y, p0=guess, bounds=bounds)
  # calcolo il coefficiente R2 per valutare la bontà del fit
  residuals = y- damped_sine(t,popt[0],popt[1],popt[2],popt[3],popt[4])
  ss_res = np.sum(residuals**2)
  ss_tot = np.sum((y-np.mean(y))**2)
  r_squared = 1 - (ss_res / ss_tot)

  if plot_guess:
    plt.plot(t, damped_sine(t, guess[0], guess[1], guess[2],guess[3],guess[4]),'--', color=[0.3,0.7,0], label="Guess", linewidth=2) # plot guess

                                                                                          
  #plot dei risultati
  
  plt.scatter(t,y,s=20,label="Exp.")     
  plt.plot(t,damped_sine(t,popt[0],popt[1],popt[2],popt[3],popt[4]),color=[1,0,0],label="Fit ($R^2=${:.4f})".format(r_squared),linewidth=2)
  plt.xlabel("tempo (s)")
  plt.ylabel("distanza (m)")
  plt.grid(which='both')
  plt.legend()
  plt.show()
  
  
  if print_output:
    label = ["A", "omega", "phi","gamma"]
    for l, v, e in zip(label, popt, np.sqrt(np.diag(pcov))) :
      print("%10s = %9.6f +- %7.6f" % (l ,v, e))
    print("R-squared = "+str(r_squared))



La seguente cella chiama al funzione di fit e visualizza il risultato finale dell'algoritmo insieme con la funzione di prova iniziale. Inizializate correttamente i parametri stimandoli dal grafico quindi cliccate sul pulsante **Fit**. 

In [ ]:
 interact_manual.opts["manual_name"]= 'Fit'
_=interact_manual(fitplot_sin, A=BoundedFloatText(min=0, max=10, step=1e-3,description='A'),
                  omega=BoundedFloatText(min=0, max=10, step=1e-3, description='omega'),
                  gamma=BoundedFloatText(min=0, max=1, step=1e-3, value=0),
                  y0=BoundedFloatText(min=0, max=2, step=1e-3),
                  phi=FloatSlider(min=-np.pi, max=np.pi, step=1e-1, value=0),
                  print_output=Checkbox(value=True, description='Visualizza parametri fit'),
                  plot_guess=Checkbox(value=True ,description='Visualizza funzione di prova'))

### Fit della legge $T(\theta)$

#### Importa i dati

La lettura del file dati viene effettuata assumendo che siano formattati in un file di testo, con tabulazioni come separatore, con le colonne corrispondenti a 

 $$\theta\quad|\quad  T \quad|\quad\sigma(\theta)\quad|\quad\sigma(T)$$

Non caricare files con lo stesso nome, eliminare prima quelli preesistenti per evitare conflitti. <br> In caso di errori di importazione può essere utile visualizzare i caratteri nascosti del file: per farlo eseguire in una cella il comando di bash `!cat -T nome_del_file.txt`

In [ ]:
dati_periodo_ampiezza = list(files.upload().keys())[0] #chiede di caricare il file e ne estrae il nome

In [ ]:
dati=np.loadtxt(dati_periodo_ampiezza, dtype=np.float, delimiter="\t", skiprows=0)

theta=dati[:,0]
T=dati[:,1]
theta_err=dati[:,2]
T_err=dati[:,3]

plt.figure()
plt.grid()
_=plt.errorbar(theta,T,yerr=T_err, xerr=theta_err, capsize=5,fmt='o')

#### Fit

In [ ]:
#Definizione funzione di fit 
def sin2(T0,theta): 
  return T0*(1+0.25*(np.sin(theta/2))**2)

In [ ]:
# Fitting routine
# definisce la funzione che stampa il grafico e esegue il fit dei dati.  È sufficiente eseguirla una volta per salvare la funzione in memoria.

 
def fitplot_period(T0, print_output=True, plot_guess=True):

  sin2_model = odr.Model(sin2)
  fit_data = odr.RealData(theta, T, sx=theta_err, sy=T_err)
  beta0 = np.array([T0]) #array of initial guesses
  
  fit = odr.ODR(fit_data, sin2_model, beta0=beta0)
  global out
  out = fit.run()
    
  # calcolo il coefficiente R2 per valutare la bontà del fit
  residuals = T- sin2(out.beta,theta)
  ss_res = np.sum(residuals**2)
  ss_tot = np.sum((y-np.mean(y))**2)
  r_squared = 1 - (ss_res / ss_tot)
  
  sample_theta = np.linspace(0,np.amax(theta),1000)
  plt.show()
 
  plt.figure(figsize = (9, 6))
 
  

  if plot_guess:
    plt.plot(sample_theta, sin2(beta0, sample_theta),'--', color=[0.3,0.7,0], label = "Guess", linewidth=2) # plot guess

  #plotting fit result
  
  plt.errorbar(theta, T,xerr=theta_err ,yerr=T_err ,linestyle='None', capsize=5, marker='o')
  plt.plot(sample_theta, sin2(out.beta,sample_theta), color=[1,0,0], label="Fit ($R^2=${:.4f})".format(r_squared), linewidth=3)
  plt.xlabel(r"$\theta$ (rad)")
  plt.ylabel("Periodo T (s)")
  plt.grid(which='both')
  plt.legend()
  plt.show()
  
  
  if print_output:
    print("T0 = {:10.4f} +/- {:2.4f}".format(out.beta[0],out.sd_beta[0]))
  print("R-squared = "+str(r_squared))



La seguente cella chiama al funzione di fit e visualizza il risultato finale dell'algoritmo insieme con la funzione di prova iniziale. Inizializate correttamente i parametri stimandoli dal grafico quindi cliccate sul pulsante **Fit**. 

In [ ]:
interact_manual.opts["manual_name"]= 'Fit'
_=interact_manual(fitplot_period, T0=BoundedFloatText(min=0, max=5, step=1e-2,description='T'),
                  print_output=Checkbox(value=True, description='Visualizza parametri fit'),
                  plot_guess=Checkbox(value=True ,description='Visualizza funzione di prova'))

## Esperimento legge di Hooke

### Fit lineare

#### Importa i dati

La lettura del file dati viene effettuata assumendo che siano formattati in un file di testo, con tabulazioni come separatore, con le colonne corrispondenti a 

 $$\text{Peso}\quad|\quad\Delta y \quad|\quad\sigma(\text{Peso})\quad|\quad\sigma(\Delta y)$$

Non caricare files con lo stesso nome, eliminare prima quelli preesistenti per evitare conflitti. In caso di errori di importazione può essere utile visualizzare i caratteri nascosti del file: per farlo eseguire in una cella il comando di bash `!cat -T nome_del_file.txt`

In [ ]:
dati_m_elongazione = list(files.upload().keys())[0] #chiede di caricare il file e ne estrae il nome

In [ ]:
dati = np.loadtxt(dati_m_elongazione, dtype=np.float, delimiter="\t", skiprows=0)

Peso = dati[:,0]
x = dati[:,1]
Peso_err = dati[:,2]
x_err = dati[:,3]

plt.figure()
plt.grid()
_ = plt.errorbar(x,Peso,yerr=Peso_err, xerr=x_err, capsize=5,fmt='o')

In [ ]:
#Definizione funzione di fit 
def retta(p,elongazione): 
  return p[0]+p[1]*elongazione

In [ ]:
# Fitting routine
# definisce la funzione che stampa il grafico e esegue il fit dei dati.  È sufficiente eseguirla una volta per salvare la funzione in memoria.

 
def fitplot_retta(y0, k, print_output=True, plot_guess=True):

  retta_model = odr.Model(retta)
  fit_data = odr.RealData(x, Peso, sx=x_err, sy=Peso_err)
  beta0 = np.array([y0,k]) #array of initial guesses
  
  fit = odr.ODR(fit_data, retta_model, beta0=beta0)
  global out
  out = fit.run()
  
  #calcolo il coefficiente R2 per valutare la bontà del fit
  residuals = Peso- retta(out.beta, x)
  ss_res = np.sum(residuals**2)
  ss_tot = np.sum((Peso-np.mean(Peso))**2)
  r_squared = 1 - (ss_res / ss_tot)
  
  sample_x = np.linspace(0,np.amax(x),1000)
  plt.show()
 
  plt.figure(figsize=(9, 6))
 
  

  if plot_guess:
    plt.plot(sample_x, retta(beta0,sample_x),'--', color=[0.3,0.7,0], label="Guess", linewidth=2) # plot guess

  #plotting fit result
  
  plt.errorbar(x, Peso, xerr=x_err, yerr=Peso_err, linestyle='None', capsize=5, marker='o')
  plt.plot(sample_x, retta(out.beta, sample_x), color=[1,0,0], label="Fit ($R^2=${:.4f})".format(r_squared), linewidth=3)
  plt.xlabel("Elongazione (m)")
  plt.ylabel("Peso (N)")
  plt.grid(which='both')
  plt.legend()
  plt.show()
  
  
  if print_output:
    print("y_0 = {:10.4f} +/- {:2.4f}".format(out.beta[0],out.sd_beta[0]))
    print("k = {:10.4f} +/- {:2.4f}".format(out.beta[1],out.sd_beta[1]))
    print("R-squared = "+str(r_squared))

 
 


La seguente cella chiama al funzione di fit e visualizza il risultato finale dell'algoritmo insieme con la funzione di prova iniziale. Inizializate correttamente i parametri stimandoli dal grafico quindi cliccate sul pulsante **Fit**. 

In [ ]:
interact_manual.opts["manual_name"]= 'Fit'
_=interact_manual(fitplot_retta, y0=BoundedFloatText(min=0, max=5, step=1e-3,description='P_0'),
                  k=BoundedFloatText(min=0, max=25, step=1e-3,description='k'),
                  print_output=Checkbox(value=True, description='Visualizza parametri fit'),
                  plot_guess=Checkbox(value=True ,description='Visualizza funzione di prova'))

### Fit sinusoidale

#### Importa i dati

In [ ]:
filename = list(files.upload().keys())[0] #chiede di caricare il file e ne estrae il nome

In [ ]:
dati = np.loadtxt(filename, dtype=np.str, delimiter="\t", skiprows=2) #legge il file formattato con tabulazioni e scarta le prime due righe di header
dati = np.char.replace(dati, ',', '.') #rimpiazza il separatore decimale
dati = dati[:,0:2]  #seleziono solo le colonne posizione e tempo
#dati[dati == ''] = 0 #rimpiazzo le stringhe vuote con 0; le trovo alla fine del file quando salvo più misure nello stesso file.
dati = dati.astype(np.float)   #riconverte le entrate a float

t = dati[:,0]
y = dati[:,1]

plt.figure()    #visualizzo i dati importati
_ = plt.plot(t,y, marker='o')

#### Fit

In [ ]:
#Definizione della funzione di fit 
def damped_sine(t,A,omega,phi,gamma,y0): 
  return A*np.exp(-gamma*t)*np.sin(omega*t+phi)+y0

In [ ]:
# Fitting routine
# definisce la funzione che stampa il grafico e esegue il fit dei dati.  È sufficiente eseguirla una volta per salvare la funzione in memoria.

 
def fitplot_sin(A, omega, y0, phi, gamma, print_output=True, plot_guess=False):
 
  guess = np.array([A,omega,phi,gamma,y0]) #array delle stime iniziali
  bounds = ([0,0,-np.pi,0,0], [10., 25, np.pi,1,10]) 
 
  global popt, pcov    #i risultati del fit sono variabili globali 
 
  plt.figure(figsize=(15, 10))
  popt, pcov = curve_fit(damped_sine, t, y, p0=guess, bounds=bounds)

# calcolo il coefficiente R2 per valutare la bontà del fit
  residuals = y- damped_sine(t,popt[0],popt[1],popt[2],popt[3],popt[4])
  ss_res = np.sum(residuals**2)
  ss_tot = np.sum((y-np.mean(y))**2)
  r_squared = 1 - (ss_res / ss_tot) 

  if plot_guess:
    plt.plot(t, damped_sine(t, guess[0], guess[1], guess[2],guess[3],guess[4]),'--', color=[0.3,0.7,0], label="Guess", linewidth=2) # plot guess

                                                                                          
  #plot dei risultati
  
  plt.scatter(t,y,s=20,label="Exp.")     
  plt.plot(t,damped_sine(t,popt[0],popt[1],popt[2],popt[3],popt[4]),color=[1,0,0],label="Fit ($R^2=${:.4f})".format(r_squared),linewidth=2)
  plt.xlabel("tempo (s)")
  plt.ylabel("distanza (m)")
  plt.grid(which='both')
  plt.legend()
  plt.show()
  
  
  if print_output:
    label = ["A", "omega", "phi","gamma"]
    for l, v, e in zip(label, popt, np.sqrt(np.diag(pcov))) :
      print("%10s = %9.6f +- %7.6f" % (l ,v, e))
    print("R-squared = "+str(r_squared))



La seguente cella chiama al funzione di fit e visualizza il risultato finale dell'algoritmo insieme con la funzione di prova iniziale. Inizializate correttamente i parametri stimandoli dal grafico quindi cliccate sul pulsante **Fit**. 

In [ ]:
 interact_manual.opts["manual_name"]= 'Fit'
_=interact_manual(fitplot_sin, A=BoundedFloatText(min=0.1, max=5, step=1e-2,description='A'),
                  omega=BoundedFloatText(min=0.1, max=20, step=1e-2, description='omega'),
                  gamma=BoundedFloatText(min=0, max=0.5, step=1e-2, value=0),
                  y0=BoundedFloatText(min=0.1, max=10, step=1e-2),
                  phi=FloatSlider(min=-np.pi, max=np.pi, step=1e-1, value=0),
                  print_output=Checkbox(value=True, description='Visualizza parametri fit'),
                  plot_guess=Checkbox(value=True ,description='Visualizza funzione di prova'))